In [3]:
import sys
sys.path.append('/Users/vdk/GeneveWork/Code/calibpipe-main-calibpipe/')
from astropy.io import ascii
import numpy as np
import pandas as pd
import matplotlib
import csv
import scipy
import matplotlib.pyplot as plt
from seaborn import histplot
import random
import copy
from os import path, makedirs
from astropy.coordinates import SkyCoord
from astropy.table import Table
import astropy.units as u
from ctapipe.containers import MuonEfficiencyContainer
from ctapipe.coordinates import CameraFrame, TelescopeFrame
from ctapipe.io import EventSource, EventSeeker
from ctapipe.core import TelescopeComponent
from ctapipe.core.traits import (
    List,
    Int,
    FloatTelescopeParameter,
    TelescopeParameter,
    Unicode,
)
from ctapipe.image.cleaning import tailcuts_clean
from ctapipe.image.muon import MuonRingFitter, MuonIntensityFitter
from traitlets.config import Config
from ctapipe.image import ImageProcessor
from ctapipe.image.muon import MuonProcessor
from ctapipe.visualization import CameraDisplay
from ctapipe.instrument import CameraGeometry
from ctapipe.calib import CameraCalibrator
from traitlets.config.loader import Config, FileConfigLoader, JSONFileConfigLoader
import pathlib
from calibpipe.throughput import MuonCalibrator
from traitlets.config import Config
from astropy.time import Time
from astropy.coordinates import EarthLocation,SkyCoord, AltAz 
import time
from ctapipe.image import number_of_island_sizes
from ctapipe.image import (
    hillas_parameters,
    leakage_parameters,
    concentration_parameters,
)
from ctapipe.image import timing_parameters
from ctapipe.image import number_of_islands
from ctapipe.image import camera_to_shower_coordinates
from lstchain.io import replace_config, standard_config

from astropy.table import Table
from astropy.io import fits
import seaborn as sns
import matplotlib.pyplot as plt
import glob
import pandas as pd
from python_modules.muonpipe import usefull_func
import json
size_outside = 400 # phe

SyntaxError: invalid syntax. Perhaps you forgot a comma? (2396958020.py, line 1)

In [4]:
files = glob.glob("/Users/vdk/Software/code/muon_cleaning_2024/clean15_muons16464/cleanmuons_16464/muons*")
files = glob.glob("/Users/vdk/muons2024/muon_cleaning_2024/clean15_muons16464/second_try/muons*")
files

['/Users/vdk/muons2024/muon_cleaning_2024/clean15_muons16464/second_try/muons_LST-1.Run16464.0004.fits',
 '/Users/vdk/muons2024/muon_cleaning_2024/clean15_muons16464/second_try/muons_LST-1.Run16464.0008.fits',
 '/Users/vdk/muons2024/muon_cleaning_2024/clean15_muons16464/second_try/muons_LST-1.Run16464.0009.fits',
 '/Users/vdk/muons2024/muon_cleaning_2024/clean15_muons16464/second_try/muons_LST-1.Run16464.0005.fits',
 '/Users/vdk/muons2024/muon_cleaning_2024/clean15_muons16464/second_try/muons_LST-1.Run16464.0002.fits',
 '/Users/vdk/muons2024/muon_cleaning_2024/clean15_muons16464/second_try/muons_LST-1.Run16464.0003.fits',
 '/Users/vdk/muons2024/muon_cleaning_2024/clean15_muons16464/second_try/muons_LST-1.Run16464.0000.fits',
 '/Users/vdk/muons2024/muon_cleaning_2024/clean15_muons16464/second_try/muons_LST-1.Run16464.0001.fits',
 '/Users/vdk/muons2024/muon_cleaning_2024/clean15_muons16464/second_try/muons_LST-1.Run16464.0006.fits',
 '/Users/vdk/muons2024/muon_cleaning_2024/clean15_muons

In [5]:
all_data_df = pd.DataFrame()
df_good_data = pd.DataFrame()
for filename in files:
    dat = Table.read(filename)
    df = dat.to_pandas()
    # Apply a couple of basic quality cuts taken from Ruben's work
    #all_data_df = df[(df['ring_containment'] > 0.999) & (df['size_outside'] < size_outside) & df['good_ring'] & (df['muon_efficiency'] < 1)]
    # Append the DataFrame to the main DataFrame
    all_data_df = df
    df_good_data= pd.concat([df_good_data, all_data_df], ignore_index=True)
    

In [6]:
df_good_data

,event_id,event_time,mc_energy,ring_size,size_outside,ring_center_x,ring_center_y,ring_radius,ring_width,good_ring,...,impact_parameter,impact_x_array,impact_y_array,radial_stdev,radial_skewness,radial_excess_kurtosis,num_pixels_in_ring,mean_pixel_charge_around_ring,hg_peak_sample,lg_peak_sample


In [8]:
filename = '/Users/vdk/Software/code/muon_cleaning_2024/clean15_muons16464/cleanmuons_16464/muons_LST-1.Run16464.0012.fits'
filename = '/Users/vdk/muons2024/muon_cleaning_2024/clean15_muons16464/second_try/muons_LST-1.Run16464.0000.fits'
dat = Table.read(filename)

In [9]:
dat

event_id,event_time,mc_energy,ring_size,size_outside,ring_center_x,ring_center_y,ring_radius,ring_width,good_ring,muon_efficiency,ring_containment,ring_completeness,impact_parameter,impact_x_array,impact_y_array,radial_stdev,radial_skewness,radial_excess_kurtosis,num_pixels_in_ring,mean_pixel_charge_around_ring,hg_peak_sample,lg_peak_sample
float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64,float64


In [16]:
filename = '/Users/vdk/muons2024/20240131muons/muons_LST-1.Run16445.0000.fits'
dat = Table.read(filename)
dat

event_id,event_time,mc_energy,ring_size,size_outside,ring_center_x,ring_center_y,ring_radius,ring_width,good_ring,muon_efficiency,ring_containment,ring_completeness,impact_parameter,impact_x_array,impact_y_array,radial_stdev,radial_skewness,radial_excess_kurtosis,num_pixels_in_ring,mean_pixel_charge_around_ring,hg_peak_sample,lg_peak_sample
int64,float64,int64,float32,float32,float64,float64,float64,float64,bool,float64,float64,float64,float64,float64,float64,float64,float64,float64,int64,float64,int64,int64
1040,1706732515.5019305,-1,2333.3308,0.0,-0.6948290333621051,-0.40932388333297426,1.2115813205885841,0.07291294108527259,True,0.16478825200250374,1.0,0.7333333333333333,7.545605203054743,4.433676533898965,6.105626116388103,0.056380552178662835,0.04339186104126951,0.1410489303129494,507,0.12211184782140395,9,0
2692,1706732515.7375839,-1,1449.5212,0.0,1.284470167193175,-1.2472656707587277,1.1247852569549512,0.06212514495534772,True,0.1421422661297232,0.5525873624363385,0.5333333333333333,7.974927556789106,-1.859895809679928,-7.755014965373651,0.05325867193188869,0.3167864534333203,-0.05149441637175434,245,0.5222555704162892,11,0
2831,1706732515.7567995,-1,1930.162,0.0,0.12583289261530922,-0.8996177787752991,1.1805404211156725,0.055091007777153374,True,0.14075578435415884,1.0,0.6,7.280162241210815,-4.6192663146506945,-5.62700106386031,0.040853808596282515,0.3480119331350728,0.48568695785859983,476,0.5048834950017472,8,0
3434,1706732515.8451636,-1,1886.8015,0.0,-1.4771091868110309,0.09693846586568058,1.1493562755479645,0.09424777249751874,True,0.18752928429670707,0.6580278931820024,0.7,2.457622237687561,1.0463848907525546,2.2237323857832303,0.07753268802653097,-0.26099916324520533,0.2011625393762615,322,0.4135419957778033,9,0
5868,1706732516.195183,-1,1320.4801,0.0,-1.9470404769913192,-0.02669890647110416,1.0643342110430105,0.069271963534778,True,0.1907465799150697,0.5160232798470124,0.43333333333333335,3.5306412082154104,-2.2345747128880182,2.7335148058260454,0.06330900369919265,-0.020822615393962296,-0.44720584524673646,213,0.2735815833596622,10,0
6379,1706732516.2695014,-1,2401.0864,0.0,0.03661561022664468,0.5911329537819756,1.176747502563742,0.05994549570662135,True,0.16925223505220754,1.0,0.8,6.6889467810849945,-6.083709255512103,2.780376005971799,0.05339587066323958,-0.26803901051132367,-0.2575381404657784,481,0.18722002918183492,10,0
6813,1706732516.3334417,-1,1448.6953,0.0,1.0407887250245222,0.8025627899986141,0.9992091498143951,0.07859523636805293,True,0.16687194191159155,0.8690293988055429,0.5,8.721598948627244,0.4426868991784178,-8.710356854342516,0.06345127314884651,0.2110435369061895,-0.35554830860037434,304,0.20228297150080105,10,0
7215,1706732516.3915038,-1,1745.6174,6.205011,1.3557214363748944,0.37735943951325723,0.9338443717128797,0.08559057053912952,True,0.1698570643655928,0.8344048033982143,0.7333333333333333,6.153532215189989,-6.045029370559466,1.1504688750480991,0.06931454569791452,-0.5456017185126589,0.6859905811402647,268,0.2110826234559755,9,0
8350,1706732516.5525358,-1,1558.0745,6.497825,0.27461481864233,-0.02787444997901442,0.8539034311400246,0.11640456512899079,True,0.18366435323775548,1.0,0.5,8.40235435366192,4.941197614470042,6.7958902889361745,0.08940740362535834,-0.30042453548187914,-0.23677490052252725,254,0.6986121941561723,11,0


### Real LST data processing

In [2]:
# Треба завантажити конфіг в івент соурс
filename = f'/Users/vdk/CTA/RealLST/typical_run_data/LST-1.4.Run14948.0001.fits.fz'

int_cut = 0.3
min_pixel = 30

config_filename = '/Users/vdk/CTA/RealLST/typical_run_data/lstchain_standard_v0.10_heuristic_ff.json'
with open(config_filename) as json_file:
    config_data = json.load(json_file)

custom_config = Config(config_data)

# muon_processor_config = Config({"MuonProcessor": 
#                                 {"RingQuery": 
#                                  {"quality_criteria" : [["intensity_check",f"np.abs(parameters.intensity_ratio - 1) < {int_cut}"],
#                                                         ["ring_containment", "parameters.containment > 0.5"],
#                                                         ["ring_completeness", "parameters.completeness > 0.5"]]}}})

# muon_processor_config_2 = Config({"MuonProcessor":
#                                 {"ImageParameterQuery" :
#                                  {"quality_criteria" : [["min_pixels", f"dl1_params.morphology.n_pixels > {min_pixel}"],
#                                                         ["min_intensity", "dl1_params.hillas.intensity > 500"]]}}})

cfg = Config({
    "source_config": {
        "EventSource" : {
            "allowed_tels": [1],"max_events": 9999}},
        "PointingSource":{
            "drive_report_path": '/Users/vdk/CTA/RealLST/typical_run_data/DrivePosition_log_20231007.txt'},
        "LSTR0Corrections": {
          "calib_scale_high_gain":1.088,
          "calib_scale_low_gain":1.004,
          "drs4_pedestal_path": '/Users/vdk/CTA/RealLST/typical_run_data/drs4_pedestal.Run14937.0000.h5',
          "calibration_path": '/Users/vdk/CTA/RealLST/typical_run_data/calibration_filters_52.Run14938.0000.h5',
          "drs4_time_calibration_path": '/Users/vdk/CTA/RealLST/typical_run_data/time_calibration.Run08349.0000.h5'
      }})

config = replace_config(standard_config, muon_processor_config)
config = replace_config(config, muon_processor_config_2)
config = replace_config(config, custom_config)


NameError: name 'json' is not defined